# NeurIPS : notebook by pragnyanramtha

In [16]:
import os
import logging
import warnings
from pathlib import Path

# Data Handling & GPU Acceleration
import pandas as pd
import numpy as np
import cupy as cp

# Machine Learning Models
import xgboost as xgb
import lightgbm as lgb
import catboost as cb
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

# Utilities
import sklearn.multioutput
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tqdm.notebook import tqdm

def setup_project_environment():
    """Creates directories, and configures logging and CUDA."""
    print("--- Project Setup ---")
    directories = ['results/models', 'results/submissions']
    for directory in directories:
        os.makedirs(directory, exist_ok=True)

    # Configure Logging
    logging.basicConfig(level=logging.INFO, 
                        format='%(asctime)s - %(levelname)s - %(message)s',
                        filename='project_log.log',
                        filemode='w')
    warnings.filterwarnings('ignore', category=FutureWarning)
    
    # Verify GPU and CuPy/PyTorch Support
    try:
        cp.cuda.runtime.getDeviceCount()
        print(f"CuPy found and can access GPU.")
    except cp.cuda.runtime.CUDARuntimeError as e:
        print(f"CuPy ERROR: {e}. Ensure CUDA toolkit is compatible.")
        
    is_torch_cuda = torch.cuda.is_available()
    print(f"PyTorch CUDA Available: {is_torch_cuda}")
    if not is_torch_cuda:
        print("WARNING: PyTorch cannot find CUDA. Training will be on CPU.")
    
    print("Setup complete.\n")

# Run the setup
setup_project_environment()

--- Project Setup ---
CuPy found and can access GPU.
PyTorch CUDA Available: True
Setup complete.



## Core Data Loading and Preprocessing

This section contains the functions responsible for interacting with the raw data. We begin by creating a robust data loader that handles the large parquet files, performs the critical ADC conversion to restore the data's dynamic range, and includes error handling. We then implement functions to apply the various calibration frames, such as dark subtraction and flat-field correction, to clean the instrumental signatures from the signal.

In [17]:
def gll_score_single(y_true, y_pred_mean, y_pred_unc):
    """Calculates the Gaussian Log-Likelihood for a single prediction."""
    return -0.5 * (np.log(2 * np.pi) + np.log(y_pred_unc**2) + ((y_true - y_pred_mean)**2) / (y_pred_unc**2))

def calculate_weighted_gll(y_true, y_pred_mean, y_pred_unc, fgs1_weight=57.846):
    """Calculates the final weighted GLL score for the competition."""
    y_true = np.asarray(y_true)
    y_pred_mean = np.asarray(y_pred_mean)
    y_pred_unc = np.asarray(y_pred_unc)
    
    scores = gll_score_single(y_true, y_pred_mean, y_pred_unc)
    
    weights = np.ones(y_true.shape[1])
    weights[0] = fgs1_weight
    
    weighted_scores = scores * weights
    return np.sum(weighted_scores)

## Comprehensive Feature Engineering Pipeline

With the data loaded and cleaned, we now focus on feature engineering. The strategy is to reduce the dimensionality of the vast time-series data into a compact and informative feature vector. We perform simple aperture photometry to create 1D light curves, extract basic statistical features from these light curves, and combine them with the scaled stellar parameters to form the final input for our models.

In [18]:
def load_adc_info(data_path):
    """Loads ADC conversion parameters, handling instrument-specific keys."""
    adc_df = pd.read_csv(Path(data_path) / 'adc_info.csv')
    return adc_df.iloc[0]

def load_and_move_to_gpu(file_path, instrument):
    """Loads a parquet file and immediately moves its data to the GPU."""
    df = pd.read_parquet(file_path)
    raw_signal_gpu = cp.asarray(df.to_numpy())
    shape = (-1, 32, 32) if instrument == 'FGS1' else (-1, 32, 356)
    return raw_signal_gpu.reshape(shape)

def apply_adc_conversion_gpu(signal_gpu, adc_params, instrument):
    """Applies ADC conversion on the GPU using instrument-specific keys."""
    gain = adc_params[f"{instrument}_adc_gain"]
    offset = adc_params[f"{instrument}_adc_offset"]
    return (signal_gpu / gain + offset).astype(cp.float64)

def apply_calibrations_gpu(signal_gpu, calib_data_gpu):
    """Applies a simplified calibration pipeline on the GPU."""
    processed_signal_gpu = signal_gpu
    if 'dark' in calib_data_gpu:
        processed_signal_gpu -= calib_data_gpu['dark']
    if 'flat' in calib_data_gpu:
        flat_gpu = calib_data_gpu['flat']
        flat_gpu[flat_gpu == 0] = 1
        processed_signal_gpu /= flat_gpu
    return processed_signal_gpu

def create_light_curve_gpu(signal_data_gpu):
    """Creates a 1D light curve on the GPU by summing pixel values."""
    return cp.sum(signal_data_gpu, axis=(1, 2)) if signal_data_gpu.ndim == 3 else cp.array([])

def extract_temporal_features_gpu(light_curve_gpu):
    """Extracts basic statistical features on the GPU and returns them to the CPU."""
    if light_curve_gpu.size == 0:
        return {'mean': 0.0, 'std': 0.0, 'min': 0.0, 'max': 0.0}
    return {
        'mean': cp.asnumpy(cp.mean(light_curve_gpu)),
        'std': cp.asnumpy(cp.std(light_curve_gpu)),
        'min': cp.asnumpy(cp.min(light_curve_gpu)),
        'max': cp.asnumpy(cp.max(light_curve_gpu))
    }

def process_planet_gpu(planet_dir, adc_params, star_info_df):
    """Main GPU processing function for a single planet."""
    planet_path = Path(planet_dir)
    planet_id = int(planet_path.name)
    features = {'planet_id': planet_id}

    for instrument in ['FGS1', 'AIRS-CH0']:
        all_visits_gpu = []
        for visit_file in sorted(planet_path.glob(f'{instrument}_signal_*.parquet')):
            visit_id = visit_file.stem.split('_')[-1]
            signal_gpu = load_and_move_to_gpu(visit_file, instrument)
            signal_gpu = apply_adc_conversion_gpu(signal_gpu, adc_params, instrument)
            
            calib_path = planet_path / f"{instrument}_calibration_{visit_id}"
            calib_data_gpu = {
                ctype: cp.asarray(pd.read_parquet(f).to_numpy())
                for ctype in ['dark', 'flat']
                if (f := calib_path / f"{ctype}.parquet").exists()
            }
            
            all_visits_gpu.append(apply_calibrations_gpu(signal_gpu, calib_data_gpu))

        if all_visits_gpu:
            full_signal_gpu = cp.concatenate(all_visits_gpu, axis=0)
            light_curve_gpu = create_light_curve_gpu(full_signal_gpu)
            temp_features = extract_temporal_features_gpu(light_curve_gpu)
        else:
            temp_features = extract_temporal_features_gpu(cp.array([]))
        
        for key, val in temp_features.items():
            features[f'{instrument}_{key}'] = val

    stellar_params = star_info_df.loc[star_info_df['planet_id'] == planet_id].iloc[0]
    features.update(stellar_params.to_dict())
    return features

## Baseline Models and Evaluation

This section establishes our modeling and evaluation framework. We implement the competition-specific Gaussian Log-Likelihood (GLL) metric, ensuring the heavy FGS1 channel weight is correctly applied. We then create a function to train a simple baseline model, such as Ridge regression, and a validation function to score its performance using a standard train-test split.

In [19]:
def train_and_evaluate_boosting_gpu(X, y):
    """Trains and evaluates GPU-accelerated XGBoost, LightGBM, and CatBoost models."""
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
    print("\n--- Training GPU-Accelerated Boosting Models ---")

    models = {
        "XGBoost": xgb.XGBRegressor(tree_method='gpu_hist', n_estimators=1000),
        "LightGBM": lgb.LGBMRegressor(device='gpu', n_estimators=1000),
        "CatBoost": cb.CatBoostRegressor(task_type='GPU', iterations=1000, verbose=0)
    }
    
    for name, model in models.items():
        print(f"\nTraining {name}...")
        multi_output_model = sklearn.multioutput.MultiOutputRegressor(model)
        multi_output_model.fit(X_train, y_train)
        
        y_pred_mean = multi_output_model.predict(X_val)
        unc = np.std(y_train - multi_output_model.predict(X_train), axis=0)
        score = calculate_weighted_gll(y_val, y_pred_mean, unc)
        print(f"{name} Validation GLL: {score:.4f}")

class SimpleMLP(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, 256), nn.ReLU(), nn.Dropout(0.3),
            nn.Linear(256, 512), nn.ReLU(), nn.Dropout(0.3),
            nn.Linear(512, output_dim)
        )
    def forward(self, x): return self.network(x)

def train_and_evaluate_pytorch_model(X, y, epochs=25, batch_size=128):
    """Orchestrates the training and evaluation of a PyTorch MLP."""
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"\n--- Training PyTorch Model on {device} ---")

    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
    train_loader = DataLoader(TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.float32)), batch_size=batch_size, shuffle=True)
    
    model = SimpleMLP(X.shape[1], y.shape[1]).to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    
    for epoch in range(epochs):
        model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X.to(device))
            loss = criterion(outputs, batch_y.to(device))
            loss.backward()
            optimizer.step()
        if (epoch + 1) % 5 == 0:
            print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.6f}")

    model.eval()
    with torch.no_grad():
        y_pred_mean_nn = model(torch.tensor(X_val, dtype=torch.float32).to(device)).cpu().numpy()
        train_preds = model(torch.tensor(X_train, dtype=torch.float32).to(device)).cpu().numpy()
    
    unc_nn = np.std(y_train - train_preds, axis=0)
    score_nn = calculate_weighted_gll(y_val, y_pred_mean_nn, unc_nn)
    print(f"PyTorch MLP Validation GLL: {score_nn:.4f}")


## Executing the Pipeline

Finally, this is the main execution script. It orchestrates the entire process by calling the functions defined above in the correct sequence. It iterates through all training planets, applies the data processing and feature engineering steps, aggregates the results into a single dataset, and then trains and evaluates our baseline model.

In [ ]:
BASE_PATH = Path('/kaggle/input/ariel-data-challenge-2025')
TRAIN_PATH = BASE_PATH / 'train'
PROCESSED_FEATURES_FILE = "processed_features.parquet"

# --- PART 1: Run Preprocessing (only if needed) ---
if not Path(PROCESSED_FEATURES_FILE).exists():
    print(f"'{PROCESSED_FEATURES_FILE}' not found. Running one-time preprocessing...")
    
    adc_params = load_adc_info(BASE_PATH)
    star_info_df = pd.read_csv(BASE_PATH / 'train_star_info.csv')
    planet_dirs = [d for d in TRAIN_PATH.iterdir() if d.is_dir()]
    
    all_features = [
        process_planet_gpu(p_dir, adc_params, star_info_df)
        for p_dir in tqdm(planet_dirs, desc="GPU Preprocessing Planets")
    ]
    
    feature_df = pd.DataFrame(all_features)
    feature_df.to_parquet(PROCESSED_FEATURES_FILE, index=False)
    print(f"\nPreprocessing complete. Features saved to '{PROCESSED_FEATURES_FILE}'.")
else:
    print(f"Found existing '{PROCESSED_FEATURES_FILE}'. Skipping preprocessing.")

# --- PART 2: Run Model Training ---
print("\n--- Starting Model Training Stage ---")

# Load data
feature_df = pd.read_parquet(PROCESSED_FEATURES_FILE)
ground_truth_df = pd.read_csv(BASE_PATH / 'train.csv')

# Merge and prepare for ML
merged_df = pd.merge(feature_df, ground_truth_df, on='planet_id', how='inner')
feature_cols = [col for col in feature_df.columns if col != 'planet_id']
target_cols = [col for col in ground_truth_df.columns if col.startswith('wl_')]

X = merged_df[feature_cols].values
y = merged_df[target_cols].values

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train all models
train_and_evaluate_boosting_gpu(X_scaled, y)
train_and_evaluate_pytorch_model(X_scaled, y)

print("\n--- Full Pipeline Finished ---")

'processed_features.parquet' not found. Running one-time preprocessing...


GPU Preprocessing Planets:   0%|          | 0/1100 [00:00<?, ?it/s]